## Матстатистика - Домашнее задание
Задача 4

In [65]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
from scipy import stats
%matplotlib inline

In [25]:
data = pd.read_csv('Data.txt', sep="\s+")
data.head()

,X1,X2,X3,X4,Y
0,6,5,8,10,7
1,0,1,2,3,0
2,10,5,6,10,1
3,7,3,8,9,8
4,10,5,7,9,4


а) Построить модель линейной регрессии Y на X1,X2,X3,X4

In [26]:
Y = data['Y'].values
X = data.iloc[:,:4].values # переводим столбцы в матрицы

reg = LinearRegression().fit(X, Y) # инициализируем модель линейной регрессии

In [55]:
def regression_coef(model, X, y, N, preds):
    # создаём датафрейм, в котором одна колонка - название коэффициентов, другая - их величина
    coef_list = [model.intercept_]
    for i in range(len(model.coef_)):
        coef_list.append(model.coef_[i])
    coef = pd.DataFrame({'predictor':preds, 
                         'coef':coef_list})
    
    MSE = metrics.mean_squared_error(y, model.predict(X))
    Var = MSE/(N-1)

    sd = np.sqrt(Var)
    t = np.array(coef_list)/ sd
    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (N - 1))) for i in t]
    coef['is_significant'] = coef['pvalue'] < 0.05
    return coef

In [56]:
regression_coef(reg, X, Y, len(Y), ['intercept', 'X1', 'X2', 'X3', 'X4'])

,predictor,coef,pvalue,is_significant
0,intercept,-0.084681,0.831554,False
1,X1,-0.083832,0.833216,False
2,X2,0.580891,0.152518,False
3,X3,0.226566,0.570325,False
4,X4,0.259176,0.516574,False


In [61]:
print('Регрессионная прямая (МНК) Y на X1,X2,X3,X4 будет иметь вид:')
print('Y=',reg.intercept_, '+', reg.coef_[0],'X1', '+', reg.coef_[1],'X2', '+', reg.coef_[2],'X3', '+', reg.coef_[3],'X4')
RSS_1 = metrics.mean_squared_error(Y, reg.predict(X))
print('RSS равен:',RSS_1)
print('Var исправленная равна:', RSS_1/(len(Y)-1))
print('R^2 равен:', reg.score(X,Y))

Регрессионная прямая (МНК) Y на X1,X2,X3,X4 будет иметь вид:
Y= -0.08468064838652545 + -0.08383160512464581 X1 + 0.5808913894916059 X2 + 0.22656637039900562 X3 + 0.25917571687868457 X4
RSS равен: 4.038644066025206
Var исправленная равна: 0.15533246407789256
R^2 равен: 0.35292933535552184


б) Определить, какую из компонент лучше всего выбрать в качетсве главной объясняющей компоненты и, рассматривая парную регрессию с ней, вычислить коэффициенты по МНК и объяснить их значимость

В модели множественной регрессии ни один коэффициент не является значимым. Однако коэффициент при X2 обладает наибольшим по модулю значением и наименьшим p-value для t-test. Следовательно, X2 вносит наобольший вес в объяснение Y.

In [83]:
reg_X2 = LinearRegression().fit(data['X2'].values.reshape(-1,1), Y)
regression_coef(reg_X2, data['X2'].values.reshape(-1,1), Y, len(Y), ['intercept', 'X2'])

,predictor,coef,pvalue,is_significant
0,intercept,2.39687,0.000003,True
1,X2,0.74679,0.078908,False


На уровне значимости 5 % коэффициент предиктора не является значимым, однако значение p-value близко к 5 % (сократилось более, чем в 2 раза по сравнению с предыдущим пунктом). 

In [63]:
print('Регрессионная прямая (МНК) Y на X2 иметь вид:')
print('Y=',reg_X2.intercept_, '+', reg_X2.coef_[0],'X2')
RSS_2 = metrics.mean_squared_error(Y, reg_X2.predict(data['X2'].values.reshape(-1,1)))
print('RSS равен:',RSS_2)
print('Var исправленная равна:', RSS_2/(len(Y)-1))
print('R^2 равен:', reg_X2.score(data['X2'].values.reshape(-1,1),Y))

Регрессионная прямая (МНК) Y на X2 иметь вид:
Y= 2.396869983948635 + 0.7467897271268059 X2
RSS равен: 4.3350127816419946
Var исправленная равна: 0.1667312608323844
R^2 равен: 0.3054452048753814


c) RSS, R^2 и Var расчитаны выше